In [1]:
using Complementarity

In [26]:
ld0 = .6 #labor demand
ls0 = .4 #Leisure demand
le0 = ld0 + ls0 #Labor endowment
fixed_le0 = le0 #We modify le0 for the shock, so store it to not change important info

kd0 = .4 #captial demand
ke0 = kd0 #Captial endowment

sigma = .7 #Elasticity of substitution in production
theta = ld0/(ld0+kd0) #Labor value share
alpha = 1/(1+ls0) #Consumption value share

0.7142857142857143

In [27]:
function little_model()

    m = MCPModel()

    @variables(m,begin
        PL>=0, (start = 1,) #Wage rate
        PK>=0, (start = 1,) #Return to capital
        PY>=0, (start = 1,) #Output price
        Y>=0,  (start = 1,) #Production
    end)


    @NLexpression(m,C,(theta*PL^(1-sigma)+(1-theta)*PK^(1-sigma))^(1/(1-sigma)))

    @NLexpression(m,AY,1)
    @NLexpression(m,AL,-ld0*(C/PL)^sigma)
    @NLexpression(m,AK,-kd0*(C/PK)^sigma)

    @NLexpression(m,DY,alpha*(PL*le0+PK*ke0)/PY)
    @NLexpression(m,DL,(1-alpha)*(PL*le0+PK*ke0)/PL)
    @NLexpression(m,DK,0)

    @NLexpression(m,BY,0)
    @NLexpression(m,BL,le0)
    @NLexpression(m,BK,ke0)



    @mapping(m,profit, -PY*AY-PL*AL-PK*AK)

    @mapping(m,marketY,BY+AY*Y - DY)
    @mapping(m,marketK,BK+AK*Y - DK)
    @mapping(m,marketL,BL+AL*Y - DL)

    @complementarity(m,profit,Y)
    @complementarity(m,marketY,PY)
    @complementarity(m,marketK,PK)
    @complementarity(m,marketL,PL)


    return m
end


little_model (generic function with 1 method)

# Benchmark

If the model is valid, all variables should have a value of 1. Which they do.

In [28]:
le0 = fixed_le0
benchmark = little_model()

solveMCP(benchmark, cumulative_iteration_limit = 0)
print(generate_report(benchmark))

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_EAAC.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 5.5511e-17           I 0.0e+00 5.6e-17 (marketL)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 5.551115e-17
var_name	 value		 margin
Y		 1.0		 0.0
PY		 1.0		 0.0
PK		 1.0		 0.0
PL		 1.0		 0.0


# Shock

Modify one of the variables, in this case we'll increase le0 by a factor of 1.1. This will throw the data out of balance. The solutions represent a relative change in value. In this solution the prices for return to caption increase by a factor of 1.0787

In [30]:
le0 = 1.1*fixed_le0

shock = little_model()

fix(shock[:PY],1,force=true)
solveMCP(shock)

print(generate_report(shock))

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 3

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             6.8881e-02             0.0e+00 (marketL)
    1     1     0     3 2.9781e-03  1.0e+00    0.0e+00 (marketL)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.9781e-03           I 0.0e+00 2.6e-03 (marketL)
    1     1     3     3 5.9909e-06  1.0e+00 SO 0.0e+00 5.6e-06 (marketL)
    2     1     4     4 2.8799e-11  1.0e+00 SO 0.0e+00 2.6e-11 (marketL)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 2.879857e-11
Postsolved residual: 2.87